Welcome to your navigation filter notebook. Here we will walk you through the steps to create a navigation filter using filterpy and spice.

Google Colab already has many Python libraries installed but there will be cases where we will not find exactly what we need.

Worry not, my dear friends, we will show you how.

We can add data from our google drive:

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

To install a library from git-hub, we use the following commands:

In [0]:
!git clone https://<user>:<password>@github.com/stardust-r/LTW-I.git

As propagator for the trajectories, we will use spice.prop2b(). A 2-body dynamics function from spice, that will be sufficient for what we want to achive. Also, we will be using spice to retrieve data for our environment modelling.

In [0]:
!pip install spiceypy

Now, the filter library that we will also be using is called filterpy, and we can install it using pip:


In [0]:
!pip install filterpy

Okay! Now that all our libraries are installed, lets start with the imports.

In [0]:
import numpy as np
from numpy.linalg import norm
from datetime import datetime
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
import spiceypy as spice
from filterpy.kalman import (
    UnscentedKalmanFilter,
    MerweScaledSigmaPoints,
    JulierSigmaPoints,
)
from filterpy.common import Q_discrete_white_noise

and a couple of modules I give you for free to ease some things you might want to do :)

In [0]:
import sys
sys.path.append('/content/LTW-I/AI_navigation/UKF/')
# Considering your module contains a function called my_func, you could import it:
from astroPlot import MakeComparisonPlot, MakeResidualsPlot, AddComparisonToPlot, AddResidualsToPlot
from astroTransf import Kep2Cart, Inertial2Hill

First, since we are using SPICE, we need to load data for celestial body charactertics and other kernels the library needs (I won't bore you with the details).

To do that, check that the metakernels paths are pointing to the right files.

In [0]:
# load spice kernels
PATH_METAKERNEL = ???
spice.furnsh(PATH_METAKERNEL)

Some constants...

In [0]:
KM2M = 1e3
M2KM = 1e-3

Now we can start setting up our simulation. We chose the asteroid Lutetia for this exercise, because the images we used in the morning were based on it. First, we define the initial state of our spacecraft.

Let's say we start in a random orbit, some 15-radii away from the asteroid. Consider that we need to be far enough to see the limb of the asteroid, or else, our ML algorithm will not work.

In [0]:
# state definition
sma = max(spice.bodvrd("LUTETIA", "RADII", 3)[1]) * 15 * KM2M  # play with this
oe = [sma, 0.1, 45, 15, 135, 77]  # random orbit
mu = spice.bodvrd("LUTETIA", "GM", 1)[1][0] * KM2M ** 3
cartState = Kep2Cart(oe, mu)

# epoch definition
tStart = datetime.now()
epoch = spice.str2et(str(tStart))

Notice the comment "# play with this". It will appear in lines where we want you to play a bit with the numbers to understand the effects of those parameters.



Because we are running a simulation, we need to create our own observations. To do that we will have 2 spacecrafts: the true one, and the estimated one.

The true SC represent what happens in reality and we should pretend that we actually dont know its position (that's what would happen in reality).

The estimated SC represents where we think the true SC is. This is our knowledge of reality. To create a gap between both, we introduce some noise on top of the true position of the spacecraft to get our estimated initial state.

In [0]:
# spacecraft True Trajectory
epochs, statesTrue = [], []
statesTrue.append(cartState)
epochs.append(epoch)

# add uncertainties for simulation
np.random.seed(11)
posStd = 1e4  # m   # play with this
velStd = 1e-1  # m / s  # play with this
state_std = np.array([posStd, posStd, posStd, velStd, velStd, velStd])
stateEst = cartState + (np.random.rand(6) * state_std * 2) - state_std

# spacecraft Sim Trajectory
statesEst = []
statesEst.append(stateEst)

We define propagate and an observations generator.

In [0]:
def propagate(???):

    # propagate using spice.prop2b
    ???

    return newState


def computeAzel(???):

    # compute azimuth and elevation of Lutetia from the SC. You can assume that
    # the sc's attitude is inertially fixed and aligned with J2000

    return azel

Apart from modelling the environment, we also need to model how our instruments perform.

This is a direct input from the performances of the algorithm we developed in the morning. If we know the pixel accuracy in the centroid detection, we should also be able to obtain the angle accuracy, right?

![](https://media.giphy.com/media/DHqth0hVQoIzS/giphy.gif)

In [0]:
# design observations
dt = 300  # s   # play with this
nObs = 10
z_std = ???  # rads - initial observervation uncertainty  # play with this

# time tags for observations
time = np.arange(0, nObs * dt, dt)

Now we create our UKF! I will give you the points and the Q matrix, but you have to initialise the filter yourselves.

In [0]:
# create sigma points to use in the filter.
points = MerweScaledSigmaPoints(6, alpha=0.1, beta=2.0, kappa=-1)

# azel observations
kf = UnscentedKalmanFilter(
    dim_x=???, dim_z=???, dt=???, fx=propagate, hx=computeAzel, points=points
)
# initial observation uncertainty
kf.R = ???

kf.x = ???  # estimated initial state
kf.P *= ???  # initial state uncertainty
# add noise to the update step
kf.Q = Q_discrete_white_noise(dim=3, dt=dt, var=0.01 ** 2, block_size=2)


We are now ready to run the sequential process for the filter. Show me what you've got.

You might want to check [this](https://nbviewer.jupyter.org/github/rlabbe/Kalman-and-Bayesian-Filters-in-Python/blob/master/10-Unscented-Kalman-Filter.ipynb) out ;)

In [0]:
obsTrue, obsEst = [], []
counter = 0

for _ in time:

    counter += 1
    print(counter)

    ???
    ???
    ???
    ???
    ???
    ???
    ???
    ???
    ???
    ???
    ???
    ???
    ???

Once the process is over, we compute some results to assess the estimation accuracy of our filter.

In [0]:
# diff in states
hillDiff = Inertial2Hill(np.array(statesTrue).T, np.array(statesEst).T) * M2KM

# residuals
residuals = (np.array(obsTrue) - np.array(obsEst)).T

And we make nice plots :)

In [0]:
axes1, fig1 = MakeComparisonPlot(
    "Truth vs Estimation",
    xlabel="Elapsed seconds",
    ylabel=["R (km)", "T (km)", "N (km)"],
)
AddComparisonToPlot(axes1, np.array(epochs) - epochs[0], hillDiff[0:3])

axes2, fig2 = MakeResidualsPlot(
    "Residuals",
    xlabel="Elapsed seconds",
    resSize=2,
    ylabel=["Az (rad)", "El (rad)"],
)
AddResidualsToPlot(
    axes2, np.array(epochs[1:]) - epochs[1], residuals, resSize=2
)

plt.show()